# Chapter 21. Asynchronous Programming

Three major topics that are related:
 - Python's `async def`, `await`, `async with`, and `async for`
 - Objects supporting those constructs: native coroutines and asynchronous variants of context managers, iterables, generators, and comprehensions
 - `asyncio` and other async libraries


## A Few Definitions

Native coroutine
 - a coroutine fcn defined with `async def`. You can delegate from a native coroutine to another native coroutine using the `await` keyword, similar to `yield from`. The `await` keyword cannot be used outside of a native coroutine.

Classic coroutine
 - A generator function that consumes datat sent to it via `my_coro.send(data)` calls, and reads that data by using `yield` in an expression. Classic coroutines can delegate to other classic coroutines using `yield from`.

Generator-based coroutine
 - A generator fcn decorated with `@types.coroutine`. That decorator makes the generator compatible with the new `await` keyword.

Asynchronous generator
 - A generator fcn defined with `async def` and using `yield` in its body. It returns an async generator object that provides `_anext_`, a coroutine method to retrieve the next itme.

10:09 - 10:25

In [ ]:
# blogdom.py

#!/usr/bin/env pythono3
import asyncio
import socket
from keyword import kwlist

MAX_KEYWORD_LEN = 4

async def probe(domain: str) -> tuple[str, bool]:
  # get a reference to the asyncio event loop
  # so we can use it next
  loop = asyncio.get_running_loop()
  try:
    # loop.getaddrinfo coroutine method
    # returns a five-part tuple of paramters
    await loop.getaddrinfo(domain, None)
  except socket.gaierror:
    return (domain, False)
  return (domain, True)

# main must be a coroutine so that we can use await in it
async def main() -> None:
  names = (kw for kw in kwlist if len(kw) <= MAX_KEYWORD_LEN)
  domains = (f'{name}.dev'.lower() for name in names)
  # build a list of coroutine objs by invoking
  # the probe coroutine with each domain argument
  coros = [probe(domain) for domain in domains]
  # asyncio.as_completed is a generator that yields
  # coroutines that return the results of the coroutine
  # passed to it **in the order they are completed**
  for coro in asyncio.as_completed(coros):
    # await expression will not block
    # but this is required for us to get the res from coro
    domain, found = await coro
    mark = '+' if found else ' '
    print(f'{mark} {domain}')

if __name__ == '__main__':
  # asyncio.run starts the event loop
  # and returns only when the event loop exits
  asyncio.run(main())

In [1]:
!python blogdom.py

+ not.dev
+ as.dev
+ from.dev
  is.dev
  with.dev
  for.dev
+ def.dev
  or.dev
  none.dev
  pass.dev
  if.dev
+ in.dev
  elif.dev
+ and.dev
+ try.dev
+ true.dev
  else.dev
+ del.dev


## Awaitable

The `for` keyword works with `iterables`. The `await` keyword works with `awaitable`.

As the end user of `asyncio`, these are awaitables we'll see frequently:
 - A native coroutine object, which you get by calling a native coroutine function
 - An `asyncio.Task` which we usually get by passing a coroutine obj to `asyncio.create_task()`

Lower-level awaitable:
 - An object with an `__await__` method that returns an iterator; for example, an `asyncio.Future` instance

 - Objects written in ohter languages using the Python/C API with a `tp_as_async.am_await` function returning an iterator

## Downloading with asyncio and HTTPX

As of Python 3.10, `asyncio` only supports TCP and UDP directly, and there are no async HTTP client or server packages in the standard library. So, we use HTTPX.

In [ ]:
# flags_asyncio.py

import asyncio

from httpx import AsyncClient

from flags import BASE_URL, save_flag, main

# must be a native coroutine
# so it can await on get_flag which does http request
# Then, it displays the code of the downloaded flag,
# and save the result
async def download_one(client: AsyncClient, cc: str):
  image = await get_flag(client, cc)
  save_flag(image, f'{cc}.gif')
  print(cc, end=' ', flush=True)
  return cc

async def get_flag(client: AsyncClient, cc: str) -> bytes:
  url = f"{BASE_URL}/{cc}/{cc}.gif".lower()
  # get method of httpx.AsyncClient returns a ClientResponse obj
  # that is also an async context manager
  resp = await client.get(url, timeout=6.1, follow_redirects=True)
  return resp.read()

# This fcn needs to be a plain fcn--not a coroutine
def download_many(cc_list: list[str]) -> int:
  # execute the event loop driving the supervisor(cc_list)
  # coroutine object until it returns
  return asyncio.run(supervisor(cc_list))

async def supervisor(cc_list: list[str]) -> int:
  # Async HTTP client operations in httpx are methods of AsyncClient
  # which is also an async context manager
  # a context manager with async setup and teardown methods
  async with AsyncClient() as client:
    # build a list of coroutine objects by calling the download_one
    # coroutine once for each flag to be retrieved
    to_do = [download_one(client, cc) for cc in sorted(cc_list)]
    # wait for the asyncio.gather coroutine,
    # which accepts one or more awaitable arguments
    # and waits for all of them to complete
    res = await asyncio.gather(*to_do)

  return len(res)

if __name__ == '__main__':
  main(download_many)

### The Secret of Native Coroutines: Humble Generators

A key difference between the classic coroutine and native coroutine is that there are no visible `.send()` calls or `yield` expression in the latter. Our code sits between the asyncio library and the async libraries we are using (e.g. HTTPX).

Under the hood, the `asyncio` event loop makes the `.send` calls that drive your coroutines, and your coroutines `await` on other coroutines, including library coroutines. (`await` borrows most of its implementation from `yield from`, which also makes `.send` calls to drive coroutines)



### All-or-Nothing Problem

## Asynchronous Context Managers